In [1]:
import unicodedata
import string
import re
import random
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [2]:


from torch import  optim
import numpy as np
import copy
import argparse
from Mask import create_masks


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
import matplotlib.pyplot as plt

In [4]:
USE_CUDA = False

In [5]:
PAD_token = 0
SOS_token = 1
EOS_token = 2
UNK_token = 3
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token:"UNK"}
        self.n_words = 4 # Count SOS and EOS
      
    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [6]:

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
#     s = s.lower() 
    s = re.sub(r"([.!?,])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?,ÄäÖöÜüẞß']+", r" ", s)
    return s

In [7]:
def read_langs(lang1, lang2, reverse=False):
    print("Reading lines...")

    pairs = []
    line1 = open('data/train.de', encoding='utf-8').read().strip().split('\n')
    line2 = open('data/train.en', encoding='utf-8').read().strip().split('\n')  #.splitlines()
    
    for i in range(len(line1)):
        
        pairs.append([normalize_string(line1[i]), normalize_string(line2[i])])
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [8]:
def prepare_data(lang1_name, lang2_name, reverse=False):
    input_lang, output_lang, pairs = read_langs(lang1_name, lang2_name, reverse)
    print("Read %s sentence pairs" % len(pairs))
    
    
    
    print("Indexing words...")
    for pair in pairs:
        input_lang.index_words(pair[0])
        output_lang.index_words(pair[1])

    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepare_data('ger', 'en')

# Print an example pair
print(random.choice(pairs))


Reading lines...
Read 196884 sentence pairs
Indexing words...
['Hier ist etwas Interessantes für die IT Welt Bio Silikon . Das sind Kieselalgen , die aus Silikaten entstehen .', "Here's an interesting one for the IT world bio silicon . This is a diatom , which is made of silicates ."]


In [9]:
print(pairs[13])

['Das meiste ist unerforscht , und doch gibt es Schönheiten wie diese , die uns fesseln und uns vertrauter mit ihm machen .', "It's mostly unexplored , and yet there are beautiful sights like this that captivate us and make us become familiar with it ."]


In [10]:
# Return a list of indexes, one for each word in the sentence
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variable_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    var = Variable(torch.LongTensor(indexes).view(-1, 1))
    if USE_CUDA: var = var.cuda()
    return var

def variables_from_pair(pair, input_lang, output_lang):
    input_variable = variable_from_sentence(input_lang, pair[0])
    target_variable = variable_from_sentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [11]:
def find_max_len(pair):
    result = 0
    
    
    for sents in pair:
        for item in sents:
            if len(item.split()) > result:
                
                result = len(item.split())
    return result

def find_len(element):
    result = 0
    for item in element:
        if len(item.split()) > result:
            result = len(item.split())
    return result

In [12]:
def paddingSOS(vector, max_len):
    vector = [SOS_token]+vector
    while len(vector)< max_len:
        vector.append(PAD_token)
    return vector

In [13]:
def paddingEOS(vector, max_len):
    vector = vector + [EOS_token]
    while len(vector)< max_len:
        vector.append(PAD_token)
    return vector

In [14]:
def padding_both(vector, max_len):
    vector = [SOS_token]+ vector + [EOS_token]
    while len(vector)< max_len:
        vector.append(PAD_token)
    return vector

In [15]:
def padding(vector, max_len):
    
    while len(vector)< max_len:
        vector.append(PAD_token)
    return vector

In [16]:
def print_head(lang):
    for i in range(8):
        print('input_lang ', i,' : ',lang.index2word[i])
print('input: ')
print_head(input_lang)
print('output: ')
print_head(output_lang)

input: 
input_lang  0  :  PAD
input_lang  1  :  SOS
input_lang  2  :  EOS
input_lang  3  :  UNK
input_lang  4  :  David
input_lang  5  :  Gallo
input_lang  6  :  Das
input_lang  7  :  ist
output: 
input_lang  0  :  PAD
input_lang  1  :  SOS
input_lang  2  :  EOS
input_lang  3  :  UNK
input_lang  4  :  David
input_lang  5  :  Gallo
input_lang  6  :  This
input_lang  7  :  is


In [17]:
max_len = find_max_len(pairs)+2
print('max_len: ', max_len)

max_len:  621


In [18]:
print('input: ', input_lang.name)
print('output: ', output_lang.name)

input:  ger
output:  en


In [19]:
def pair_to_indexes(pairs, max_len, input_lang, output_lang):
    source = np.zeros((len(pairs), max_len))
    target = np.zeros((len(pairs), max_len))
    for i in range(len(pairs)):
        # add start token for english
        sent2 = padding_both(indexes_from_sentence(output_lang, pairs[i][1]), max_len)
        sent2 = torch.Tensor(sent2)
        
        target[i] = sent2
        
        # add end token for german
        sent1 = padding(indexes_from_sentence(input_lang, pairs[i][0]), max_len)
        sent1 = torch.Tensor(sent1)
        source[i] = sent1
    
    return source, target

In [20]:
data_1 = [element for element in pairs if find_len(element) < 200]
print(len(data_1) / len(pairs))

0.999812072083054


In [21]:
def correspond_len(pair, thres):
    le = find_len(pair)
    if le < thres[0]-3:
        return thres[0]
    for i in range(len(thres)):
        if i == len(thres)-1:
            return None
        if le > (thres[i]-3) and le < (thres[i+1]-3):
            return thres[i+1]

In [22]:
def class_data(data_pairs):
    threshold = [20, 40, 60, 80, 100, 200]
    class_pairs = []
    for i in range(len(threshold)):
        class_pairs.append([])
    for pair in data_pairs:
        pair_len = correspond_len(pair, threshold)
        if pair_len is None:
            continue
        class_pairs[threshold.index(pair_len)].append(pair)
        
    return class_pairs, threshold
    

In [23]:
class_pairs, thres = class_data(data_1)


In [24]:
for i in range(6):
    print(len(class_pairs[i]))

98382
71526
14677
2791
619
269


In [25]:
print(output_lang.name, output_lang.n_words)
print(input_lang.name, input_lang.n_words)

en 58313
ger 122633


In [31]:
from Model import Transformer
params = {}


params['batch_size'] = 2
params['epochs'] = 30
params['learning_rate'] = 0.001


dim_model = 300
H = 12
N = 6
src_vocab = input_lang.n_words
trg_vocab = output_lang.n_words

model = Transformer(src_vocab, trg_vocab, dim_model, N, H)



## TODO: 1. add bleu score for each epoch 2. change datas to pairs 

In [32]:



# dataset: pairs
def train_lm(data_pairs, params, net):
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    
    
    classed_pairs, thres = class_data(data_pairs)
    sources_set = []
    targets_set = []
    batches_set = []
    for i in range(len(classed_pairs)):
        source, target = pair_to_indexes(classed_pairs[i], thres[i], input_lang, output_lang)
        sources = torch.from_numpy(source)
        targets = torch.from_numpy(target)
    
        num_examples = len(classed_pairs[i])
        batches = [(start, start + params['batch_size']) for start in \
               range(0, num_examples, params['batch_size'])]
        sources_set.append(sources)
        targets_set.append(targets)
        batches_set.append(batches)

    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        
        # for each batch, calculate loss and optimize model parameters
        for i in range(len(batches_set)):
            batches = batches_set[i]
            random.shuffle(batches)
            sources = sources_set[i]
            targets = targets_set[i]
            for b_idx, (start, end) in enumerate(batches):
                source = sources[start:end]
                target = targets[start:end]

                source_mask, target_mask = create_masks(source, target)
                preds = net(source, target, source_mask, target_mask)


                preds = preds[:, :-1, :].contiguous().view(-1, net.target_vocab)
                labels = target[:, 1:].contiguous().view(-1)
                loss = criterion(preds, labels.long())
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                ep_loss += loss.item()

        curr_loss = 'epoch: %d, loss: %0.2f, time: %0.2f sec' % (epoch, ep_loss, time.time() - start_time)
        print(curr_loss)

        
            



In [ ]:

 #[20, 40, 60, 80, 100, 200]
# for i in range(len(data_1)):
#     if find_len(data_1[i]) > 600:
#         print('100', i)
#     elif find_len(data_1[i]) > 80:
#         print('80', i)
#     elif find_len(data_1[i]) > 60:
#         print('60',i)
#     elif find_len(data_1[i]) > 40:
#         print('40',i)
#     elif find_len(data_1[i]) > 20:
#         print('20',i)
    
# train_lm(data_1, params, model)

In [29]:
data_temp = data_1[:5]
data_temp.append(data_1[14])
data_temp.append(data_1[16])
data_temp.append(data_1[30])
data_temp.append(data_1[36])
data_temp.append(data_1[1240])
data_temp.append(data_1[1478])
data_temp.append(data_1[56354])
data_temp.append(data_1[55323])
data_temp.append(data_1[57843])
data_temp.append(data_1[58482])

In [33]:
train_lm(data_temp, params, model)

epoch: 0, loss: 180.72, time: 10.61 sec
epoch: 1, loss: 166.62, time: 10.03 sec
epoch: 2, loss: 99.96, time: 9.98 sec
epoch: 3, loss: 77.18, time: 10.03 sec
epoch: 4, loss: 65.42, time: 10.12 sec
epoch: 5, loss: 55.51, time: 9.88 sec
epoch: 6, loss: 55.32, time: 10.25 sec
epoch: 7, loss: 54.64, time: 10.04 sec
epoch: 8, loss: 44.67, time: 9.97 sec
epoch: 9, loss: 40.43, time: 9.96 sec
epoch: 10, loss: 38.98, time: 10.06 sec
epoch: 11, loss: 40.85, time: 10.21 sec
epoch: 12, loss: 40.37, time: 10.06 sec
epoch: 13, loss: 37.21, time: 10.02 sec
epoch: 14, loss: 38.52, time: 9.94 sec
epoch: 15, loss: 39.24, time: 10.12 sec
epoch: 16, loss: 33.70, time: 10.45 sec
epoch: 17, loss: 32.74, time: 10.91 sec
epoch: 18, loss: 32.85, time: 10.13 sec
epoch: 19, loss: 30.18, time: 10.09 sec
epoch: 20, loss: 29.53, time: 10.07 sec
epoch: 21, loss: 31.22, time: 10.07 sec
epoch: 22, loss: 28.62, time: 10.78 sec
epoch: 23, loss: 24.38, time: 10.07 sec
epoch: 24, loss: 20.82, time: 10.07 sec
epoch: 25, lo

In [34]:
train_lm(data_temp, params, model)

epoch: 0, loss: 24.12, time: 10.53 sec
epoch: 1, loss: 19.99, time: 10.15 sec
epoch: 2, loss: 21.23, time: 10.58 sec
epoch: 3, loss: 22.57, time: 10.24 sec
epoch: 4, loss: 17.16, time: 10.08 sec
epoch: 5, loss: 15.31, time: 10.00 sec
epoch: 6, loss: 14.73, time: 10.02 sec
epoch: 7, loss: 11.65, time: 10.05 sec
epoch: 8, loss: 10.22, time: 10.12 sec
epoch: 9, loss: 9.12, time: 10.17 sec
epoch: 10, loss: 8.33, time: 10.19 sec
epoch: 11, loss: 7.62, time: 10.16 sec
epoch: 12, loss: 7.69, time: 10.09 sec
epoch: 13, loss: 7.31, time: 10.00 sec
epoch: 14, loss: 6.11, time: 10.09 sec
epoch: 15, loss: 5.75, time: 10.15 sec
epoch: 16, loss: 6.23, time: 10.01 sec
epoch: 17, loss: 5.11, time: 10.09 sec
epoch: 18, loss: 3.84, time: 10.05 sec
epoch: 19, loss: 3.40, time: 10.05 sec
epoch: 20, loss: 3.33, time: 10.15 sec
epoch: 21, loss: 2.49, time: 10.32 sec
epoch: 22, loss: 1.83, time: 10.59 sec
epoch: 23, loss: 2.29, time: 10.09 sec
epoch: 24, loss: 2.08, time: 10.25 sec
epoch: 25, loss: 1.67, tim

## restore model from .pt file

## translate

In [35]:
def traslante_sentence(curr_sent, max_len_1, input_lang, output_lang):
    source, target = pair_to_indexes(curr_sent, max_len_1, input_lang, output_lang)
    target_fake = np.zeros((1, max_len_1))
    target_fake[0][0] = 1
    target_temp = target_fake
    
    for i in range(max_len_1-2):
        sou = torch.from_numpy(source)
        tar = torch.from_numpy(target_fake)
        source_mask, target_mask = create_masks(sou, tar)
        preds = model(sou, tar, source_mask, target_mask)

        preds = preds[:, :-1,:].contiguous().view(-1, model.target_vocab)
        ss = torch.softmax(preds, dim=-1)
        mm = torch.max(ss, dim=-1)[1]
        target_temp[0][i+1] = mm[i]
        target_fake = target_temp
    result = ''
    for idx in mm:
        if idx == 0:
            break
        index = idx.item()
        if index == 2:
            break
        result += output_lang.index2word[index]+' '
    print(result)



In [36]:
data_curr = pairs[2:3]
print(data_curr)
print(find_len(data_curr[0]))


[['Wir haben ein paar der unglaublichsten Aufnahmen der Titanic , die man je gesehen hat , , und wir werden Ihnen nichts davon zeigen .', "We've got some of the most incredible video of Titanic that's ever been seen , and we're not going to show you any of it ."]]
26


In [37]:
traslante_sentence(data_curr, 40, input_lang, output_lang)

We've got some of the most incredible video of Titanic that's ever been seen , and we're not going to show you any of it . 


In [31]:
# import bcolz
# import pickle
# vectors = bcolz.open(f'emb/6B.300.dat')[:]
# words = pickle.load(open(f'emb/6B.300_words.pkl', 'rb'))
# word2idx = pickle.load(open(f'emb/6B.300_idx.pkl', 'rb'))

# eng = {w: vectors[word2idx[w]] for w in words}

# vectors = bcolz.open(f'emb/ger.300.dat')[:]
# words = pickle.load(open(f'emb/ger.300_words.pkl', 'rb'))
# word2idx = pickle.load(open(f'emb/ger.300_idx.pkl', 'rb'))

# ger = {w: vectors[word2idx[w]] for w in words}

In [32]:
# print(len(eng['hello']))

In [33]:
# print(output_lang.name, output_lang.n_words)
# print(input_lang.name, input_lang.n_words)

In [34]:
# import numpy as np
# input_emb = np.zeros((input_lang.n_words, 300))
# for i in range(input_lang.n_words):
#     curr_word = input_lang.index2word[i]
#     try:
#         vec = ger[curr_word.lower()]
#         input_emb[i] = vec
#     except KeyError:
#         vec = torch.rand((1, 300))
#         input_emb[i] = vec

In [35]:
# # # save
# np.save('emb/ger', input_emb)


In [36]:
# input_reload = np.load('emb/ger.npy')
# print(input_reload[20][:10])

[-0.160972  0.006209 -0.008173 -0.484031  0.067782  0.242032  0.437631
 -0.121384 -0.247417  0.112258]


In [37]:
# output_emb = np.zeros((output_lang.n_words, 300))
# for i in range(output_lang.n_words):
#     curr_word = output_lang.index2word[i]
#     try:
#         vec = eng[curr_word.lower()]
#         output_emb[i] = vec
#     except KeyError:
#         vec = torch.rand((1, 300))
#         output_emb[i] = vec

In [38]:
# # save
# np.save('emb/eng', output_emb)
# output_reload = np.load('emb/eng.npy')

In [39]:
# print(eng['world'][:10])
# print(output_lang.word2index['world'])
# print(output_emb[141][:10])
# print(output_reload[141][:10])

[-0.25831   0.43644  -0.1138   -0.5259    0.20213   0.95247  -0.58764
 -0.047001 -0.053704 -1.744   ]
141
[-0.25831   0.43644  -0.1138   -0.5259    0.20213   0.95247  -0.58764
 -0.047001 -0.053704 -1.744   ]
[-0.25831   0.43644  -0.1138   -0.5259    0.20213   0.95247  -0.58764
 -0.047001 -0.053704 -1.744   ]
